In [19]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/quebec-mrc-urban-agglomeration-2023-uds/pf-mun-2023-2023.csv
/kaggle/input/quebec-mrc-urban-agglomeration-2023-uds/pf-mun-2023-2022.csv
/kaggle/input/quebec-mrc-urban-agglomeration-2023-uds/pf-postes-2023-2023.csv


**Dataset Overview**

This notebook utilizes a dataset provided by Données Québec, a rich source of public data. For additional details and resources, please visit their official website: [Données Québec](http://https://www.donneesquebec.ca/exploiter/).

**The primary aim of this workbook is twofold:**

1.     Data Cleaning: We will embark on a journey to clean the dataset meticulously. This process is crucial as it ensures the data's quality, making it suitable for further analysis and insights generation.

2.     Demonstrating Python's Role in Hybrid Workflows: Through this exercise, we aim to showcase how Python can serve as a powerful tool in hybrid workflows. Specifically, we will illustrate Python's utility in preparing data for analysis in tools like PowerBI, thereby highlighting its versatility in both standalone and integrated scenarios.

By the end of this workbook, you will gain insights into the practical applications of Python in data preparation and its effectiveness in enhancing data analysis workflows.

In [20]:
# imports
# install pandas using pip or mamba/conda
# import pandas as pd
# import OS

In [21]:
# generate the dataframes
# we are importing the definition of poste to merge the column name in df_mun_2023_2023 to make the columns human readable 
df_def_poste = pd.read_csv('/kaggle/input/quebec-mrc-urban-agglomeration-2023-uds/pf-postes-2023-2023.csv')
df_mun_2023_2023 = pd.read_csv('/kaggle/input/quebec-mrc-urban-agglomeration-2023-uds/pf-mun-2023-2023.csv')

# output the head for each dataframe to get to know the data we are using
print(df_def_poste.head())
print(df_mun_2023_2023.head())

  groupe      poste                                        DESCRIPTION
0     CP  PFRAT0111  RFU par unité d'évaluation imposable et compen...
1     CP    PRC1410  Évaluation imposable uniformisée résidentielle...
2     CP    PRC1510  Évaluation imposable uniformisée industrielle ...
3     CP    PRC1610  Évaluation imposable uniformisée agricole / Év...
4     CP    PRC2010        Évaluation moyenne uniformisée par logement
   an_profil  an_donnee cod_geo_n               nom_mun designation  \
0       2023       2023     AG010  Îles-de-la-Madeleine          AG   
1       2023       2023     AG230                Québec          AG   
2       2023       2023     AG410       Cookshire-Eaton          AG   
3       2023       2023     AG582             Longueuil          AG   
4       2023       2023     AG660              Montréal          AG   

   population classe_population cod_mrc nom_mrc  cod_ra  ... IND0102 RFU3002  \
0       13189   10 000 @ 24 999     NaN     NaN      11  ...     Na

In [22]:
# Create a dictionary from df_def_poste where the keys are 'poste' and the values are 'DESCRIPTION'
poste_description_dict = df_def_poste.set_index('poste')['DESCRIPTION'].to_dict()

# cycle through poste_description_dict and change the column names in df_mun_2023_2023
for key in poste_description_dict:
    if key in df_mun_2023_2023.columns:
        df_mun_2023_2023.rename(columns={key: poste_description_dict.get(key, key)}, inplace=True)

In [23]:
# Setting up df for splitting into metro area
df_mun_2023_2023 = df_mun_2023_2023.fillna({'cod_mrc': "Agglo", 'nom_mrc': "Agglo"})
# output a table to show the change
print(df_mun_2023_2023[df_mun_2023_2023['cod_mrc'] == "Agglo"].head())

   an_profil  an_donnee cod_geo_n               nom_mun designation  \
0       2023       2023     AG010  Îles-de-la-Madeleine          AG   
1       2023       2023     AG230                Québec          AG   
2       2023       2023     AG410       Cookshire-Eaton          AG   
3       2023       2023     AG582             Longueuil          AG   
4       2023       2023     AG660              Montréal          AG   

   population classe_population cod_mrc nom_mrc  cod_ra  ...  \
0       13189   10 000 @ 24 999   Agglo   Agglo      11  ...   
1      593056   100 000 et plus   Agglo   Agglo       3  ...   
2        6273     2 000 @ 9 999   Agglo   Agglo       5  ...   
3      437683   100 000 et plus   Agglo   Agglo      16  ...   
4     2048125   100 000 et plus   Agglo   Agglo       6  ...   

  Indice RFU par unité d'évaluation imposable et compensable  \
0                                                NaN           
1                                                NaN        

In [24]:
# generate a new df for every cod_mrc that has the value Agglo
agglo = df_mun_2023_2023[df_mun_2023_2023['cod_mrc'] == 'Agglo']

# Cleaning empty rows in  agglo df
agglo = agglo.drop(columns=['cod_cm', 'nom_cm', 'Richesse foncière uniformisée (RFU)', 'Évaluation imposable uniformisée résidentielle / Évaluation uniformisée des immeubles imposables', 'Évaluation imposable uniformisée industrielle et commerciale / Évaluation uniformisée des immeubles imposables', 'Évaluation imposable uniformisée agricole / Évaluation uniformisée des immeubles imposables', 'Évaluation imposable uniformisée autre / Évaluation uniformisée des immeubles imposables', "Évaluation moyenne uniformisée des résidences d'un logement (incluant condominiums)", 'Évaluation moyenne uniformisée par logement', "RFU par unité d'évaluation imposable et compensable", "Indice RFU par unité d'évaluation imposable et compensable", "Nombre d'unité d'évaluation imposable et compensable", 'Évaluation uniformisée des immeubles imposables', 'Évaluation imposable uniformisée résidentielle', 'Évaluation imposable uniformisée industrielle et commerciale', 'Évaluation imposable uniformisée agricole'])

# store the number of rows in the dataframe in a variable
num_rows_mun_2023 = df_mun_2023_2023.shape[0]

# Drop rows df_mun_2023_2023 with NaN values in 'cod_mrc' column
df_mun_2023_2023 = df_mun_2023_2023[df_mun_2023_2023['cod_mrc'] != 'Agglo']

# update the new amount of rows in num_rows_mun_2023 variable 
num_rows_mun_2023 = df_mun_2023_2023.shape[0]

#show the number of rows before and after our operation
print(f"number of rows: {num_rows_mun_2023}")
print(f"number of rows now: {df_mun_2023_2023.shape[0]}")


number of rows: 1104
number of rows now: 1104


In [25]:
# create a new df called metro_area_qc 
metro_area_qc = df_mun_2023_2023[df_mun_2023_2023['cod_cm'].notna()]

# clean metro area columns
metro_area_qc = metro_area_qc.drop(columns=['Évaluation imposable uniformisée industrielle et commerciale - agglomération', 'Évaluation imposable uniformisée résidentielle - agglomération', 'Évaluation uniformisée des immeubles imposables - agglomération', "Nombre d'unité d'évaluation imposable et compensable - agglomération", "RFU par unité d'évaluation imposable et compensable - agglomération", 'Richesse foncière uniformisée (RFU) - agglomération', 'Évaluation imposable uniformisée autre / Évaluation uniformisée des immeubles imposables - agglomération', 'Évaluation imposable uniformisée agricole / Évaluation uniformisée des immeubles imposables - agglomération', 'Évaluation imposable uniformisée industrielle et commerciale / Évaluation uniformisée des immeubles imposables - agglomération', 'Évaluation imposable uniformisée résidentielle / Évaluation uniformisée des immeubles imposables - agglomération', 'Évaluation imposable uniformisée autre - agglomération', 'Évaluation imposable uniformisée agricole - agglomération'])

# drop the cols in df_mun_2023_2023 based on code cm since they we're the only ones that were not empty
df_mun_2023_2023 = df_mun_2023_2023[df_mun_2023_2023['cod_cm'].isna()]

#show the number of rows before and after our operation
print(f"number of rows: {num_rows_mun_2023}")
print(f"number of rows now: {df_mun_2023_2023.shape[0]}")

number of rows: 1104
number of rows now: 995


In [26]:
# output all df to the output folder
metro_area_qc.to_csv('/kaggle/working/metro_area_qc.csv', index=False)
agglo.to_csv('/kaggle/working/agglo.csv', index=False)
df_mun_2023_2023.to_csv('/kaggle/working/df_mun_2023.csv', index=False)

# output the head for each dataframe to make sure file are non empty
print(metro_area_qc.head())
print(agglo.head())
print(df_mun_2023_2023.head())

# Moving to powerbi for the next steps
# Follow my project here: https://github.com/jcardibo/TaxeFonciereQC_2023_PowerBI


     an_profil  an_donnee cod_geo_n                            nom_mun  \
230       2023       2023     20005  Saint-François-de-l'Île-d'Orléans   
231       2023       2023     20010  Sainte-Famille-de-l'Île-d'Orléans   
232       2023       2023     20015      Saint-Jean-de-l'Île-d'Orléans   
233       2023       2023     20020   Saint-Laurent-de-l'Île-d'Orléans   
234       2023       2023     20025    Saint-Pierre-de-l'Île-d'Orléans   

    designation  population classe_population cod_mrc          nom_mrc  \
230           M         573         1 @ 1 999   AR200  L'Île-d'Orléans   
231           M         921         1 @ 1 999   AR200  L'Île-d'Orléans   
232           M        1196         1 @ 1 999   AR200  L'Île-d'Orléans   
233           M        1780         1 @ 1 999   AR200  L'Île-d'Orléans   
234           M        2009     2 000 @ 9 999   AR200  L'Île-d'Orléans   

     cod_ra  ...  \
230       3  ...   
231       3  ...   
232       3  ...   
233       3  ...   
234       